In [ ]:
import pandas as pandas
import numpy as np
import matplotlib.pyplot as plt
import seaborn as snb
import os,sys
import timedate

In [ ]:
input_file_name="seattle-weather.csv"
input_file_path=os.path.join(os.getcwd(),"data",input_file_name)

df=pd.read_csv(input_file_path)